In [1]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

import ccxt as cx
from typing import Any, Optional, List
import ccxt.pro as cxp
from ccxt.base.decimal_to_precision import ROUND_UP
import asyncio
import nest_asyncio
nest_asyncio.apply()
from ccxt.base.exchange import Exchange

import qubx
%qubxd dev

%load_ext autoreload
%autoreload 2

from qubx import lookup
from qubx.core.series import TimeSeries, Bar, Trade, Quote, OHLCV
from qubx.utils.pandas import scols, ohlc_resample
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent
from qubx.core.basics import Instrument, Position, Signal, TransactionCostsCalculator, dt_64
from qubx.impl.ccxt_connector import CCXTConnector, CtrlChannel, BinanceQV
from qubx.utils.pandas import ohlc_resample

 >  [dev] installing cython rebuilding hook

                   ╻
   ┏┓      ╻       ┃  ┏┓       Quantitative Backtesting Environment 
   ┃┃  ┓┏  ┣┓  ┏┓  ┃  ┏┛       
   ┗┻  ┗┻  ┗┛  ┗   ┃  ┗━       (c) 2024,  ver. Dev
                   ╹       



# Common

In [ ]:
lookup.instruments['KRAKEN.F:BTC/USD']

In [34]:
instr = lookup.instruments.find('BINANCE.UM', 'BTC', 'USDT')
instr

Instrument(symbol='BTCUSDT', market_type='CRYPTO', exchange='BINANCE.UM', base='BTC', quote='USDT', margin_symbol='USDT', min_tick=0.1, min_size_step=0.001, min_size=0.001, futures_info=FuturesInfo(contract_type='PERPETUAL', delivery_date=datetime.datetime(2100, 12, 25, 10, 0), onboard_date=datetime.datetime(2019, 9, 25, 11, 0), contract_size=1.0, maint_margin=2.5, required_margin=5.0, liquidation_fee=0.0125), _aux_instrument=None)

In [39]:
instr = lookup.instruments.find('KRAKEN', 'BTC', 'ETH')
instr

Instrument(symbol='ETH/BTC', market_type='spot', exchange='KRAKEN', base='ETH', quote='BTC', margin_symbol=None, min_tick=1e-05, min_size_step=1e-05, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [38]:
lookup.find_aux_instrument_for(instr, 'USD')

Instrument(symbol='BTC/USD', market_type='spot', exchange='KRAKEN', base='BTC', quote='USD', margin_symbol=None, min_tick=0.1, min_size_step=0.1, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [46]:
bu = cxp.binanceusdm()
bs = cxp.binance()
ks = cxp.kraken()
kf = cxp.krakenfutures()

# Get OHLC [TODO]

In [47]:
bs_markets = await bs.load_markets()
bu_markets = await bu.load_markets()
kf_markets = await kf.load_markets()
ks_markets = await ks.load_markets()

In [53]:
bu.parse_timeframe('1m')

60

In [ ]:
interval = bu.parse_timeframe('1m') * 1000
orderbook_max_levels_limit = 100
since = bu.round_timeframe('1m', bu.milliseconds(), ROUND_UP) - orderbook_max_levels_limit * interval
await bu.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=orderbook_max_levels_limit)

In [ ]:
# await bnc1.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=limit)

In [57]:
cxc = CCXT_connector('binance')

# <font color='#ff0000'>Listen to data></font>

In [2]:
import asyncio
# from multiprocessing import Queue, Process, Event, Lock
from threading import Thread, Event, Lock
from queue import Queue
# from multiprocessing import Queue, Process, Event, Lock
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [3]:
class CtrlChannel:
    """
    Controlled data communication channel
    """
    control: Event
    queue: Queue     # we need something like disruptor here (Queue is temporary)
    name: str
    lock: Lock

    def __init__(self, name: str):
        self.name = name
        self.control = Event()
        self.queue = Queue()
        self.lock = Lock()

    def stop(self):
        if self.control.is_set():
            self.control.clear()

    def start(self):
        self.control.set()


class AsyncioThreadRunner(Thread):
    channel: Optional[CtrlChannel]

    def __init__(self, channel: Optional[CtrlChannel]):
        self.result = None
        self.channel = channel
        self.loops = []
        super().__init__()

    def add(self, func, *args, **kwargs) -> 'AsyncioThreadRunner':
        self.loops.append(func(self.channel, *args, **kwargs))
        return self

    async def run_loop(self):
        self.result = await asyncio.gather(*self.loops)

    def run(self):
        if self.channel:
            self.channel.control.set()
        asyncio.run(self.run_loop())

    def stop(self):
        if self.channel:
            self.channel.control.clear()
            self.channel.queue.put((None, None)) # send sentinel

In [37]:
class TestExch:
    def __init__(self, exch: Exchange) -> None:
        self.exchange = exch
        self.loop = asyncio.get_running_loop()

    async def listen_to_trades(self, channel: CtrlChannel, symbols: List[str]):
        print("Start watch_trades_for_symbols ...")
        while channel.control.is_set():
            try:
                trades = await self.exchange.watch_trades_for_symbols(symbols)
                for trade in trades:
                    t_ns = trade['timestamp'] * 1_000_000 # this is trade time 
                    s, info, price, amnt = trade['symbol'], trade['info'], trade['price'], trade['amount']
                    m = info['m']
                    tas = Trade(t_ns, price, amnt, int(not m), int(trade['id']))
                    channel.queue.put((s, tas))
            except Exception as e:
                print(type(e).__name__, str(e), flush=True)
                await self.exchange.close()
                raise e
        print("Stop watch_trades_for_symbols")

    async def listen_to_orderbook(self, channel: CtrlChannel, symbols: List[str], orderbook_max_levels_limit=5):
        while channel.control.is_set():
            try:
                orderbook = await self.exchange.watch_order_book_for_symbols(symbols, limit=orderbook_max_levels_limit)
                t_ns = orderbook['timestamp'] * 1_000_000
                s, b, a = orderbook['symbol'], orderbook['bids'], orderbook['asks']
                q = Quote(t_ns, b[0][0], a[0][0], b[0][1], a[0][1])
                channel.queue.put((s, q))
            except Exception as e:
                print(type(e).__name__, str(e), flush=True)
                await self.exchange.close()
                raise e

    async def fetch_ohlcs(self, symbol: str, timeframe: str, nbarsback: int):
        start = ((pd.Timestamp('now', tz='UTC') - nbarsback * pd.Timedelta(timeframe)).asm8.item()//1000000) if nbarsback > 1 else None 
        print("> START fetching OHLCV...")
        return await self.exchange.fetch_ohlcv(symbol, timeframe, since=start, limit=nbarsback + 1)

    async def receiver(self, channel: CtrlChannel):
        sers = {}
        print(">>> START LISTENING", flush=True)
        T, Q = 0, 0
        while channel.control.is_set():
            s, data = channel.queue.get()
            if isinstance(data, Quote):
                if not (Q % 100):
                    print(s, data, flush=True)
                    Q = 0
                Q += 1
            if isinstance(data, Trade):
                if s not in sers:
                    sers[s] = TimeSeries(s, '1Min')
                ts = sers[s]
                ts.update(data.time, data.price)
                if not (T % 200):
                    print(s, data, flush=True)
                    T = 0
                    print("--- Send request for OHLCV ...")
                    t1 = self.loop.create_task(self.fetch_ohlcs(s, '1m', 10))
                    while not t1.done():
                        await asyncio.sleep(0.01)
                    print("DATA: ", t1.result())
                    # print("DATA: ", data)
                T += 1
            if isinstance(data, Bar):
                # print(f"{s} {pd.Timestamp(data.time, unit='ns')}: {data}", flush=True)
                if s not in sers:
                    sers[s] = {}
                sers[s][data.time] = data

        print(f">>> STOP LISTENING: {str(sers)}", flush=True)
        return sers 

In [38]:
ctx = TestExch(BinanceQV())

In [39]:
ch = CtrlChannel('A')
rn0 = AsyncioThreadRunner(ch)
# rn1 = AsyncioThreadRunner(ch)
# rn0.add( ctx.listen_to_trades, ['BTCUSDT'])
# rn1.add( ctx.receiver)

# rn1.start()
# rn0.start()
ch.start()

In [8]:
# pool1 = ThreadPoolExecutor(3)
# pool2 = ThreadPoolExecutor(3)

In [10]:
# pool1, pool2

(<concurrent.futures.thread.ThreadPoolExecutor at 0x2281009f550>,
 <concurrent.futures.thread.ThreadPoolExecutor at 0x2283650a450>)

In [40]:
loop = asyncio.get_running_loop()
t0 = loop.create_task(ctx.listen_to_trades(ch, ['BTCUSDT']))
t1 = loop.create_task(ctx.fetch_ohlcs('BTCUSDT', '1m', 10))
rn0.add( ctx.receiver)
rn0.start()

# c2 = await loop.run_in_executor(pool2, ctx.listen_to_trades, ch, ['BTCUSDT'])
# await asyncio.gather(c1, c2)

>>> START LISTENING


Start watch_trades_for_symbols ...
> START fetching OHLCV...
BTC/USDT [2024-04-01T11:02:37.519000000]	69643.44000 (0.0) <make> 3524778593
--- Send request for OHLCV ...
> START fetching OHLCV...
DATA:  [[1711968780000, 69544.16, 69558.05, 69544.16, 69558.05, 3.44984, 239948.4872952, 151848.9324906], [1711968840000, 69558.04, 69577.14, 69551.08, 69577.13, 10.91972, 759578.8338085, 441910.6662739], [1711968900000, 69577.13, 69577.14, 69551.09, 69551.09, 10.29042, 715825.1139341, 92807.3883297], [1711968960000, 69551.09, 69575.99, 69551.09, 69561.19, 12.30624, 856011.0503375, 361478.1261965], [1711969020000, 69561.18, 69572.1, 69552.0, 69552.01, 7.51446, 522723.9835937, 133638.4159887], [1711969080000, 69552.0, 69552.01, 69552.0, 69552.0, 2.19828, 152894.7797674, 64039.3176874], [1711969140000, 69552.0, 69558.96, 69552.0, 69554.01, 6.85855, 477060.7335589, 322629.2464023], [1711969200000, 69554.0, 69558.97, 69532.12, 69558.96, 14.82266, 1030892.4702116, 610302.8618266], [1711969260000, 69

In [41]:
ch.stop()

>>> STOP LISTENING: {'BTC/USDT': 2024-04-01 11:02:00    69654.21
Name: BTC/USDT, dtype: float64}


In [94]:
# rn0.stop()
# rn1.stop()

In [4]:
def prefetch_ohlcs(exchange: Exchange, symbol: str, timeframe: str, nbarsback: int):
    assert nbarsback > 1
    start = ((pd.Timestamp('now', tz='UTC') - nbarsback * pd.Timedelta(timeframe)).asm8.item()//1000000) if nbarsback > 1 else None 
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=start, limit=nbarsback + 1)
    return ohlcv

async def listen_to_ohlcv(channel: CtrlChannel, exchange: Exchange, 
                          symbol: List[str], timeframe: str, nbarsback: int):
    # - check if we need to load initial 'snapshot'
    if nbarsback > 1:
        ohlcv = await prefetch_ohlcs(exchange, symbol, timeframe, nbarsback)
        for oh in ohlcv:
            b = Bar(oh[0] * 1_000_000, oh[1], oh[2], oh[3], oh[4], oh[6], oh[7])
            channel.queue.put((symbol, b))

    while channel.control.is_set():
        try:
            ohlcv = await exchange.watch_ohlcv(symbol, timeframe)
            for oh in ohlcv:
                b = Bar(oh[0] * 1000000, oh[1], oh[2], oh[3], oh[4], oh[6], oh[7])
                channel.queue.put((symbol, b))
        except Exception as e:
            print(type(e).__name__, str(e), flush=True)
            await exchange.close()
            raise e

In [5]:
symbs = ['1INCHUSDT', 'AGLDUSDT', 'APEUSDT', 'ARBUSDT', 'AUCTIONUSDT', 'BANDUSDT', 'BATUSDT', 'C98USDT', 'COMPUSDT', 'CYBERUSDT',
        'DYDXUSDT', 'ENSUSDT', 'FXSUSDT', 'HFTUSDT', 'ICXUSDT', 'IOSTUSDT', 'IOTAUSDT', 'IOTXUSDT', 'LDOUSDT', 'LQTYUSDT', 'LTCUSDT',
        'MASKUSDT', 'MBOXUSDT', 'MINAUSDT', 'MTLUSDT', 'OGNUSDT', 'OPUSDT', 'PEOPLEUSDT', 'POLSUSDT', 'PYRUSDT', 'QNTUSDT', 'RADUSDT',
        'RDNTUSDT', 'RIFUSDT', 'RLCUSDT', 'SEIUSDT', 'SPELLUSDT', 'SSVUSDT', 'STGUSDT', 'STXUSDT', 'SUIUSDT', 'UMAUSDT', 'WAVESUSDT']

In [6]:
# chan = CtrlChannel('ticks')
# reader = RunThread(chan)
# reader.add(receiver)
# reader.start()

START LISTENING


In [7]:
# bu = cxp.binanceusdm()
# bu = cxp.binance()
# bu = BinanceQV()
# bu = getattr(cxp, 'binanceqv')()
# t1 = RunThread(chan)
# for s in symbs:
#     t1.add(listen_to_ohlcv, bu, s, '15m', 10)

# t1.add(listen_to_ohlcv, bu, 'BTCUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'ETHUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'LTCUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'SOLUSDT', '5m', 10)
# t1.add(listen_to_trades, bu, ['BTCUSDT', 'ETHUSDT', 'SOLUSDT'])
# t1.add(listen_to_orderbook, bu, ['BTCUSDT', 'ETHUSDT', 'SOLUSDT'])
# t1.start()

In [10]:
reader.stop()
t1.stop()

In [ ]:
print(len(reader.result[0]), len(symbs))
reader.result[0]

# StrategyCtx

In [43]:
from qubx.core.basics import Instrument, Position, Signal
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent

In [44]:
from qubx import logger

class ZeroStrat(IStrategy):
    param1: int = 0

    def on_start(self, ctx: StrategyContext):
        logger.info(f"> Started with param1 = {self.param1}")

    def on_event(self, ctx: StrategyContext, event: TriggerEvent) -> Optional[List[Signal]]:
        logger.info(f"{event.time} -> {event}")
        xs = scols(*[ctx.ohlc(s, '1Min').pd()['close'].rename(s.symbol) for s in ctx.instruments])
        print(xs)

        return None

    def on_stop(self, ctx: StrategyContext):
        logger.info(f"> stopped")

In [45]:
cc = CCXTConnector('binance')
cc.time()

numpy.datetime64('2024-04-01T11:05:39.743101000')

In [18]:
# cc.get_historical_ohlcs('BTCUSDT', '1d', 20)

In [46]:
ctx = StrategyContext(
    ZeroStrat(), dict(param1 = 1), cc, cc, 
    instruments=lookup.instruments[r'BINANCE:(BTC|ETH|LTC)USDT'],
    capital=1000, fees_spec='vip0_usdt', base_currency='USDT',
    md_subscription=dict(type='ohlc', timeframe='1Min', nback=10),
    trigger=dict(type='bar', delay='-1s', timeframe='1Min')
)

2024-04-01 14:05:40.965 [ ℹ️ ] Set param1 -> 1


In [47]:
ctx.start()

2024-04-01 14:05:42.535 [ ℹ️ ] Subscribing to ohlc updates using {'timeframe': '1Min', 'nback': 10} for 
	['BTCUSDT', 'ETHUSDT', 'LTCUSDT'] 
2024-04-01 14:05:42.539 [ ℹ️ ] > Started with param1 = 1
2024-04-01 14:05:42.540 [ ℹ️ ] > Data processor started
2024-04-01 14:05:42.540 [ ℹ️ ] Start processing market data
2024-04-01 14:05:42.540 [ ℹ️ ] > Market data subscribtions started


In [49]:
ctx.stop()

2024-04-01 14:05:49.334 [ ℹ️ ] > stopped
2024-04-01 14:05:49.334 [ ℹ️ ] Market data processing finished


In [ ]:
ctx.ohlc('BTCUSDT', '1Min')

In [50]:
await cc.exch.close()

AttributeError: 'CCXTConnector' object has no attribute 'exch'

In [ ]:
# cc.close()

<hr/>

# Asyncs